You are tasked with creating a neural network that HR can use to predict whether employees are likely to leave the company. Additionally, HR believes that some employees may be better suited to other departments, so you are also asked to predict the department that best fits each employee. These two columns should be predicted using a branched neural network.

## Part 1: Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from keras import Model
from keras import layers
os.environ["KERAS_BACKEND"] = "tensorflow"

In [2]:
#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [3]:
# Determine the number of unique values in each column.
attrition_df.nunique()

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64

In [4]:
attrition_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 27 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Age                       1470 non-null   int64 
 1   Attrition                 1470 non-null   object
 2   BusinessTravel            1470 non-null   object
 3   Department                1470 non-null   object
 4   DistanceFromHome          1470 non-null   int64 
 5   Education                 1470 non-null   int64 
 6   EducationField            1470 non-null   object
 7   EnvironmentSatisfaction   1470 non-null   int64 
 8   HourlyRate                1470 non-null   int64 
 9   JobInvolvement            1470 non-null   int64 
 10  JobLevel                  1470 non-null   int64 
 11  JobRole                   1470 non-null   object
 12  JobSatisfaction           1470 non-null   int64 
 13  MaritalStatus             1470 non-null   object
 14  NumCompaniesWorked      

In [5]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[["Attrition", "Department"]]
y_df.head()

,Attrition,Department
0,Yes,Sales
1,No,Research & Development
2,Yes,Research & Development
3,No,Research & Development
4,No,Research & Development


In [6]:
y_department = y_df['Department']
y_attrition = y_df['Attrition']

In [7]:
# Create a list of at least 10 column names to use as X data
selected_columns = [
    "Education",
    "Age",
    "DistanceFromHome",
    "JobSatisfaction",
    "OverTime",
    "StockOptionLevel",
    "WorkLifeBalance",
    "YearsAtCompany",
    "YearsSinceLastPromotion",
    "NumCompaniesWorked"
]

# Create X_df using your selected columns
X_df = attrition_df[selected_columns]

# Show the data types for X_df
X_df.dtypes


Education                   int64
Age                         int64
DistanceFromHome            int64
JobSatisfaction             int64
OverTime                   object
StockOptionLevel            int64
WorkLifeBalance             int64
YearsAtCompany              int64
YearsSinceLastPromotion     int64
NumCompaniesWorked          int64
dtype: object

In [8]:
# Split the data into training and testing sets (80% training, 20% testing)
X_train, X_test, y_department_train, y_department_test, y_attrition_train, y_attrition_test = train_test_split(X_df, y_department, y_attrition, test_size=0.2, random_state=42)

In [9]:
# Convert your X data to numeric data types however you see fit
#Add new code cells as necessary
X_df["OverTime"].value_counts()

OverTime
No     1054
Yes     416
Name: count, dtype: int64

In [10]:
from sklearn.preprocessing import LabelEncoder

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Apply LabelEncoder to the 'OverTime' column in both the training and testing sets
X_train['OverTime'] = label_encoder.fit_transform(X_train['OverTime'])
X_test['OverTime'] = label_encoder.transform(X_test['OverTime'])

# Display the first few rows of the transformed X_train
X_train.head()

,Education,Age,DistanceFromHome,JobSatisfaction,OverTime,StockOptionLevel,WorkLifeBalance,YearsAtCompany,YearsSinceLastPromotion,NumCompaniesWorked
1097,2,24,21,1,0,3,3,1,0,0
727,2,18,5,4,0,0,3,0,0,1
254,2,29,20,4,0,1,3,3,0,2
1175,3,39,12,2,0,0,3,5,1,4
1341,3,31,20,3,0,1,3,10,0,1


In [11]:
# Initialize the StandardScaler
scaler = StandardScaler()

# Fit the scaler on the training data and transform both the training and testing sets
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Display the first few rows of the scaled training data
X_train_scaled

array([[-0.86335572, -1.38855944,  1.44039645, ..., -0.97426331,
        -0.67610953, -1.05916816],
       [-0.86335572, -2.04073779, -0.52269928, ..., -1.13857331,
        -0.67610953, -0.65943075],
       [-0.86335572, -0.84507748,  1.31770296, ..., -0.6456433 ,
        -0.67610953, -0.25969335],
       ...,
       [ 1.06322176, -1.60595222, -0.76808624, ..., -1.13857331,
        -0.67610953, -1.05916816],
       [-0.86335572, -0.84507748,  0.45884859, ..., -0.4813333 ,
        -0.67610953,  0.53978146],
       [ 0.09993302,  1.43754676, -0.03192534, ..., -0.97426331,
        -0.67610953,  0.53978146]])

In [12]:
# Create a OneHotEncoder for the Department column
# Fit the encoder to the training data
# Create two new variables by applying the encoder
# to the training and testing data

from sklearn.preprocessing import OneHotEncoder

# Initialize the OneHotEncoder
department_encoder = OneHotEncoder(sparse=False)

# Fit the encoder on the 'Department' column in the training set
department_encoder.fit(np.array(y_department_train).reshape(-1, 1))

# Transform the 'Department' column in the testing set
department_encoded_train = department_encoder.transform(np.array(y_department_train).reshape(-1, 1))
department_encoded_test = department_encoder.transform(np.array(y_department_test).reshape(-1, 1))

# Display the first few rows of the transformed training data
department_encoded_train

c:\Users\gisi_\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


array([[0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       ...,
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [13]:
# Create a OneHotEncoder for the Attrition column
attrition_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore', drop=None)
# Fit the encoder to the training data
attrition_encoder.fit(np.array(y_attrition_train).reshape(-1, 1))
# Create two new variables by applying the encoder
# to the training and testing data
attrition_encoded_train = attrition_encoder.transform(np.array(y_attrition_train).reshape(-1, 1))
attrition_encoded_test = attrition_encoder.transform(np.array(y_attrition_test).reshape(-1, 1))
attrition_encoded_train

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [0., 1.],
       [1., 0.],
       [1., 0.]])

## Create, Compile, and Train the Model

In [14]:
# # Find the number of columns in the X training data
# # Create the input layer
input_shape = (X_train_scaled.shape[1],)

# Step 1: Create the input layer using a tuple for the input shape
input_layer = layers.Input(shape=input_shape)

In [15]:
# Create at least two shared layers
shared_layer1 = layers.Dense(64, activation='relu', name ='shared1')(input_layer)
shared_layer2 = layers.Dense(128, activation='relu', name ='shared2')(shared_layer1)

In [16]:
# Create a branch for Department
# with a hidden layer and an output layer
# Create the hidden layer
# Create the output layer

department_hidden = layers.Dense(32, activation='relu', name ='department_hidden')(shared_layer2)
department_output = layers.Dense(department_encoded_train.shape[1], activation='softmax', name ='department_output')(department_hidden)

In [17]:
# Create a branch for Attrition
# with a hidden layer and an output layer
# Create the hidden layer
# Create the output layer
attrition_hidden = layers.Dense(32, activation='relu', name ='attrition_hidden')(shared_layer2)
attrition_output = layers.Dense(attrition_encoded_train.shape[1], activation='sigmoid', name ='attrition_output')(attrition_hidden)

In [18]:
# Create the model
model = Model(inputs=input_layer, 
              outputs={'department_output': department_output, 'attrition_output': attrition_output})

# Compile the model
model.compile(optimizer='adam', 
loss={'department_output': 'categorical_crossentropy', 'attrition_output': 'binary_crossentropy'},
metrics={'department_output': 'accuracy', 'attrition_output':'accuracy'})

# Summarize the model
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 10)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ shared1 (Dense)     │ (None, 64)        │        704 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ shared2 (Dense)     │ (None, 128)       │      8,320 │ shared1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attrition_hidden    │ (None, 32)        │      4,128 │ shared2[0][0]     │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ department_hidden   │ (None, 32)        │      4,128 │ shared2[0][0]     │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attrition_output    │ (None, 2)         │         66 │ attrition_hidden… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ department_output   │ (None, 3)         │         99 │ department_hidde… │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 17,445 (68.14 KB)

 Trainable params: 17,445 (68.14 KB)

 Non-trainable params: 0 (0.00 B)

In [19]:
# Train the model
model.fit(
    X_train_scaled,
    {'department_output': department_encoded_train, 'attrition_output': attrition_encoded_train},
    epochs=100,
    batch_size=32,
    validation_split=0.2
)

Epoch 1/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - attrition_output_accuracy: 0.8282 - attrition_output_loss: 0.5708 - department_output_accuracy: 0.5962 - department_output_loss: 0.9208 - loss: 1.4908 - val_attrition_output_accuracy: 0.7966 - val_attrition_output_loss: 0.5118 - val_department_output_accuracy: 0.6314 - val_department_output_loss: 0.7879 - val_loss: 1.3295
Epoch 2/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - attrition_output_accuracy: 0.8436 - attrition_output_loss: 0.4326 - department_output_accuracy: 0.6496 - department_output_loss: 0.7720 - loss: 1.2051 - val_attrition_output_accuracy: 0.7966 - val_attrition_output_loss: 0.4552 - val_department_output_accuracy: 0.6314 - val_department_output_loss: 0.7804 - val_loss: 1.2675
Epoch 3/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - attrition_output_accuracy: 0.8521 - attrition_output_loss: 0.3970 - department_output_accuracy: 0.6437 - department_output_loss: 0.7460 - loss: 1.1430 - val_attrition_output_accuracy: 0.8051 -

In [20]:
# Evaluate the model with the testing data
results = model.evaluate(X_test_scaled,
                                    {
                                        'department_output': department_encoded_test,
                                        'attrition_output': attrition_encoded_test
                                    },
                                    verbose=1)
results

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - attrition_output_accuracy: 0.8291 - attrition_output_loss: 1.4956 - department_output_accuracy: 0.5551 - department_output_loss: 2.9266 - loss: 4.4705


[4.259379863739014,
 1.3021326065063477,
 2.6914868354797363,
 0.8367347121238708,
 0.5272108912467957]

In [21]:
# Print the accuracy for both department and attrition
department_accuracy = results[4]
attrition_accuracy = results[3]

# Print the results
print(f"Department predictions accuracy: {department_accuracy * 100:.2f}%")
print(f"Attrition predictions accuracy: {attrition_accuracy * 100:.2f}%")

Department predictions accuracy: 52.72%
Attrition predictions accuracy: 83.67%


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

1. Accuracy is a useful metric for a general overview, but it may not always be the best metric, especially if there is class imbalance in the data (e.g., more employees belong to one department than others, or significantly more employees do not leave the company than do). In such cases, metrics like precision, recall, or the F1-score may provide more insight, particularly if false positives or false negatives are more important to avoid in one of the tasks (e.g., attrition prediction).

2. For the Department output, I used the softmax activation function, as it is ideal for multi-class classification tasks (outcomes are mutually exclusive). Softmax ensures that the output probabilities sum to 1, assigning each department a probability score, and the department with the highest probability is selected as the prediction. For the other hand, with the Attrition output I used the sigmoid activation function, which is suitable for binary classification. Sigmoid outputs a value between 0 and 1, making it ideal for predicting whether an employee will leave the company (0 = no, 1 = yes).


3. Here are some ways to improve the model:

* Feature Engineering: Experiment with different feature combinations or create new features that better capture the relationships between employees' attributes and their department or likelihood of leaving.

* Handling Class Imbalance: If there is class imbalance, using techniques like class weighting, oversampling, or undersampling can help the model learn better from minority classes (especially for department prediction).

* Hyperparameter Tuning: Optimize parameters such as learning rate, batch size, and the number of neurons in hidden layers to improve model performance. Grid search or random search can help find the best combination.

* Regularization: Use techniques like dropout or L2 regularization to prevent overfitting, particularly if the model performs well on the training data but poorly on the test data.

* Model Complexity: Consider increasing the complexity of the neural network by adding more layers or neurons to the shared or branch-specific layers, especially for department prediction, where accuracy is low.

